# College Basketball Code
## Dan McDonough
## February 25th, 2021

The purpose of this notebook is to compile four factors stats for D1 NCAAM teams ahead of march madness. My goal is to write functionality that gives a very condensed, stylistic game preview to digest ahead of watching the game.

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import random
import timeit
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import scipy as sp

In [2]:
combined_df = pd.read_csv('college.csv')

combined_df['FTR'] = combined_df['FTR'] * 100
combined_df['EFG'] = combined_df['EFG'] * 100
combined_df['Opp FTR'] = combined_df['Opp FTR'] * 100
combined_df['Opp EFG'] = combined_df['Opp EFG'] * 100

### 2. Write Summary Function

In [3]:
def game_summary(away,home):
    away_df = combined_df[combined_df['Team']==away]
    home_df = combined_df[combined_df['Team']==home]
    home_df_adj = home_df[['Team', 'SRS', 'SOS', 'Pace','DEF', 'Opp FTR', 'Opp EFG', 'Opp TOV', 'Opp ORB', 'OFF', 'FTR', 
                           'EFG', 'TOV', 'ORB']].copy()
    away_df.columns = ['Team', 'SRS', 'SOS', 'Pace','Away Rtg', 'Away FTR', 'Away EFG', 'Away TOV', 'Away ORB',
                       'Home Rtg', 'Home FTR', 'Home EFG', 'Home TOV', 'Home ORB']
    home_df_adj.columns = ['Team', 'SRS', 'SOS', 'Pace','Away Rtg', 'Away FTR', 'Away EFG', 'Away TOV', 'Away ORB',
                       'Home Rtg', 'Home FTR', 'Home EFG', 'Home TOV', 'Home ORB']
    output_df = pd.concat([away_df,home_df_adj],axis=0)
    means_df = pd.DataFrame(output_df.mean()).transpose()
    output_df = pd.concat([output_df,means_df],sort=False).reset_index(drop=True)
    output_df.loc[2,'Team']='Average'
    for column in output_df.columns[1:]:
        output_df[column] = output_df[column].astype(int)
    return output_df[['Team', 'SRS', 'SOS', 'Pace','Away Rtg', 'Away EFG', 'Away TOV', 'Away ORB', 'Away FTR',
                       'Home Rtg', 'Home EFG', 'Home TOV', 'Home ORB','Home FTR']]

Friday

In [4]:
game_summary('Loyola (IL)','Ohio State')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Loyola (IL),14,3,65,110,56,16,25,31,92,47,18,23,28
1,Ohio State,14,9,65,103,47,13,28,29,112,54,14,28,33
2,Average,14,6,65,107,52,14,26,30,102,50,16,26,30


Pregame: Pass at -1

Away: Shooting

Home: Shooting, Rebounding

In [5]:
game_summary('Jacksonville State','Auburn')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Jacksonville State,0,-4,67,109,55,17,29,30,98,48,15,25,21
1,Auburn,20,8,71,92,44,17,29,34,108,50,14,33,31
2,Average,10,2,69,100,49,17,29,32,103,49,14,29,26


Pregame: Pass at -15.5

Away: Shooting

Home: Rebounding, Free Throws

In [6]:
game_summary('Montana State','Texas Tech')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Montana State,0,-5,69,109,54,16,27,38,97,47,16,22,34
1,Texas Tech,19,8,67,88,45,20,26,32,105,52,17,33,36
2,Average,10,1,68,98,49,18,26,35,101,50,17,27,35


Pregame: Pass at -15

Away: Shooting, Free Throws

Home: Shooting, Rebounding

In [7]:
game_summary('Yale','Purdue')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Yale,0,-1,70,102,50,16,25,32,97,48,16,26,30
1,Purdue,19,8,66,102,49,12,23,21,118,56,14,35,36
2,Average,9,3,68,102,50,14,24,27,108,52,15,30,33


Pregame: Pass at -16

Away: Free Throws

Home: Shooting, Rebounding

In [8]:
game_summary('Delaware','Villanova')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Delaware,0,-2,68,107,53,16,27,33,101,48,16,30,28
1,Villanova,19,9,64,97,47,16,28,25,112,52,13,30,30
2,Average,9,3,66,102,50,16,27,29,107,50,14,30,29


Pregame: Pass at -15.5

Away: Shooting, Free Throws

Home: Consider betting team total

In [9]:
game_summary('Miami (FL)','Southern California')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Miami (FL),10,6,67,110,54,12,23,28,104,53,17,30,24
1,Southern California,13,6,67,97,44,12,27,26,106,51,15,33,30
2,Average,11,6,67,103,49,12,25,27,105,52,16,32,27


Pregame: USC at -1

Away: Shooting

Home: Free Throws

In [10]:
game_summary('Notre Dame','Alabama')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Notre Dame,11,6,65,109,54,13,22,26,101,49,13,24,22
1,Alabama,15,11,73,103,49,15,30,35,108,51,16,35,33
2,Average,13,9,69,106,51,14,26,31,104,50,14,30,27


Pregame: Pass at -4

Away: Shooting, Rebounding, Free Throws

Home: Rebounding, Free Throws

In [11]:
game_summary('Virginia Tech','Texas')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Virginia Tech,14,5,63,111,55,14,28,23,97,48,16,26,25
1,Texas,17,8,64,92,46,20,29,36,105,50,15,31,32
2,Average,15,7,63,101,50,17,28,30,101,49,16,28,28


Pregame: Texas at -1

Away: Shooting, Turnovers, Free Throws

Home: Rebounding, Free Throws

In [12]:
game_summary('Chattanooga','Illinois')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Chattanooga,5,-1,66,112,53,14,32,26,97,48,16,26,25
1,Illinois,17,9,68,98,46,13,25,28,110,53,15,33,31
2,Average,11,4,67,105,50,13,29,27,103,50,15,29,28


Pregame: Lean Illinois at -7.5

Away: Shooting, Rebounding

Home: Shooting, Rebounding, Free Throws

In [13]:
game_summary("Cal State Fullerton",'Duke')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Cal State Fullerton,0,-2,67,104,49,15,29,35,98,50,18,27,26
1,Duke,19,6,68,97,47,13,28,18,116,55,12,31,28
2,Average,9,1,67,101,48,14,29,27,107,53,15,29,27


Pregame: Pass at -18.5

Away: Free Throws

Home: Shooting, Turnovers

In [14]:
game_summary("Iowa State","Louisiana State")

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Iowa State,12,8,67,98,49,17,28,28,93,48,21,28,36
1,Louisiana State,18,8,70,89,45,21,31,38,102,49,17,33,33
2,Average,15,8,68,93,47,19,29,33,97,48,19,31,34


Pregame: Lean LSU at -4

Away: Free Throws

Home: Rebounding

In [15]:
game_summary('Wright State','Arizona')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Wright State,-1,-5,69,108,51,15,31,31,102,50,16,29,25
1,Arizona,24,6,72,92,44,15,28,22,115,55,15,34,35
2,Average,11,0,71,100,48,15,29,27,108,53,15,31,30


Pregame: Pass at -21.5

Away: Shooting, Free Throws

Home: Shooting, Rebounding, Free Throws

In [16]:
game_summary('UAB','Houston')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,UAB,11,0,70,113,53,13,33,28,92,47,19,27,30
1,Houston,22,5,65,89,43,18,27,35,114,53,14,37,28
2,Average,16,2,68,101,48,16,30,31,103,50,16,32,29


Pregame: Lean Houston at -8.5

Away: Shooting, Turnovers, Rebounding, Free Throws

Home: Shooting, Turnovers, Rebounding

In [17]:
game_summary('Davidson','Michigan State')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Davidson,10,1,66,113,56,13,23,32,99,49,15,21,27
1,Michigan State,13,10,68,100,47,13,27,28,105,52,16,30,30
2,Average,12,5,67,107,51,13,25,30,102,50,15,26,29


Pregame: Pass at -1

Away: Shooting

Home: Rebounding

In [18]:
game_summary('Colgate','Wisconsin')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,Colgate,0,-6,67,111,56,14,26,26,98,48,14,24,24
1,Wisconsin,13,9,67,99,49,15,24,27,105,48,11,25,32
2,Average,7,1,67,105,52,14,25,27,101,48,12,25,28


Pregame: Lean Wisconsin at -7.5

Away: Shooting

Home: Free Throws

In [19]:
game_summary('TCU','Seton Hall')

,Team,SRS,SOS,Pace,Away Rtg,Away EFG,Away TOV,Away ORB,Away FTR,Home Rtg,Home EFG,Home TOV,Home ORB,Home FTR
0,TCU,12,9,67,101,48,18,37,31,96,47,16,26,26
1,Seton Hall,13,8,68,96,45,15,29,27,105,48,14,33,31
2,Average,13,9,67,98,47,16,33,29,101,48,15,29,29


Pregame: Pass at even

Away: Rebounding

Home: Rebounding, Free Throws